# Init

In [1]:
from pyfiles import (
    complex as cplx,
    grid as grid,
    utils as utils,
    examples as ex,
    input as inp,
    plot as ourplot
)
from pyfiles.examples import CameraOpt

# so we don't have to reboot kernel every time we modify classes.py
%load_ext autoreload
%autoreload 2

import numpy as np
from typing import *
import matplotlib.pyplot as plt
%matplotlib inline
import mars

def export_obj(ex: ex.Example, complex: cplx.complex, faces: List[np.ndarray]):
    out_name = ex.filename.replace('input', 'output')
    name = out_name.split('/')[-1]
    name = name.split('.')[0]
    filename = f'output/{name}-GB{ex.grid_buffer}-GS{ex.grid_size}-MA{ex.medial_axis}'
    if ex.prune_eps is not None:
        filename += f'-peps{ex.prune_eps}'
    filename += '.obj'

    with open(filename, 'w') as f:
        vi = 1
        f.write(f'o {name}\n')
        for triangle in complex.trilist:
            a = triangle.coords[0]
            b = triangle.coords[1]
            c = triangle.coords[2]
            f.write(f'v {a[0]} {a[1]} {a[2]}\n')
            f.write(f'v {b[0]} {b[1]} {b[2]}\n')
            f.write(f'v {c[0]} {c[1]} {c[2]}\n')
            f.write(f'f {vi + 0} {vi + 1} {vi + 2}\n')
            vi += 3

        f.write(f'o {name}-MA{ex.medial_axis}\n')
        for [a, b, c, d] in faces:
            f.write(f'v {a[0]} {a[1]} {a[2]}\n')
            f.write(f'v {b[0]} {b[1]} {b[2]}\n')
            f.write(f'v {c[0]} {c[1]} {c[2]}\n')
            f.write(f'v {d[0]} {d[1]} {d[2]}\n')
            f.write(f'f {vi + 0} {vi + 1} {vi + 2} {vi + 3}\n')
            vi += 4
    print(f'Wrote {filename}')

def plotline(ax, p, q, *args, **kwargs):
    ax.plot([p[0], q[0]], [p[1], q[1]], [p[2], q[2]], *args, **kwargs)


# input

In [2]:
# example = ex.theta

# our_complex = inp.read_obj(example.filename)
# gridd = grid.Grid3.from_complex(our_complex, example.grid_size, buffer=example.grid_buffer)

# rs_complex = mars.read_from_obj(example.filename)

# list_of_swaps = []
# '''A list of swaps of the form `(old-cell, new-cell, [swaps])`'''

# rs_states = {}
# '''States for each point.  Used to live in the python Grid3'''

# n_edges = gridd.number_of_grid_edges()
# visit_count = 0
# def visit(new_cell: Tuple[int,int,int], old_cell: Tuple[int,int,int]):
#   global visit_count
#   visit_count += 1
#   perf = visit_count / n_edges * 100
#   print(f'visiting {visit_count}/{n_edges} ({perf:.1f}%)\r', end='')
#   if old_cell is None:
#     center_point = gridd.coordinate(new_cell)
#     with utils.Timed('rs reduce'):
#       rs_states[new_cell] = mars.reduce_from_scratch(rs_complex, list(center_point))
#   else:
#     new_point = gridd.coordinate(new_cell)
#     with utils.Timed('rs reduce_vine'):
#       rs_old_state = rs_states[old_cell]
#       (rs_new_state, swaps) = mars.vineyards_123(rs_complex, rs_old_state, list(new_point))
#       list_of_swaps.append((old_cell, new_cell, swaps))
#       rs_states[new_cell] = rs_new_state

# with utils.Timed('flood_fill_visit'):
#   gridd.flood_fill_visit((0, 0, 0), visit)
# utils.Timed.report()



In [3]:
example = ex.theta
rs_complex = mars.read_from_obj(example.filename)
rgrid = mars.Grid.around_complex(rs_complex, example.grid_size, example.grid_buffer)
with utils.Timed('all rust'):
  res = rgrid.run_state(1000, rs_complex, True)
utils.Timed.report()

Reduce around 3 states from scratch ...
pop grid volume 3360
pop grid volume 1800
pop grid volume 1680
pop grid volume 990
Reduce dim0 ...done
Reduce dim1done
Reduce dim2done
Invert V0 ...done
Invert V1 ...done
Invert V2 ...done
pop grid volume 900
Reduce dim0 ...done
Reduce dim1done
Reduce dim2done
Invert V0 ...done
Invert V1 ...done
Invert V2 ...done
pop grid volume 924
Reduce dim0 ...done
Reduce dim1done
Reduce dim2done
Invert V0 ...done
Invert V1 ...done
Invert V2 ...done
pop grid volume 840
Reduce dim0 ...done
Reduce dim1done
Reduce dim2done
Invert V0 ...done
Invert V1 ...done


# chop chop

In [ ]:
# CONTROL PARAMETERS
medaxdim = 0 # can say example.medial_axis when set

# euclidean prune: all dims
euclid_prune = True
prune_dist = 0.4

# coboundary: dim 0,2
cofaceprune = True

# faceprune: dim 1,2
faceprune = False

# persistence prune: dim 1
persprune = False
persistence_threshold = 0.01

###########################################

faces = []
rs_states, swaps = res
for (old_cell, new_cell, swaps) in swaps:
  state_old = rs_states[old_cell]
  state_new = rs_states[new_cell]
  swaps = swaps.pyclone()

  # PRUNING
  # coboundary: mainly for dim 0,2
  if cofaceprune:
    swaps.prune_coboundary(rs_complex)
  if faceprune:
    swaps.prune_common_face(rs_complex)

  # persistence: mainly for dim 1
  if persprune:
    swaps.prune_persistence(rs_complex, state_old, state_new, persistence_threshold)

  # euclidean prune: for all dims
  # NOTE: dist prune is NOT grid size based, it is absolute dist
  # if example.prune_dist is not None:
  #   swaps.prune_euclidian(rs_complex, example.prune_dist**2)
  if euclid_prune:
    swaps.prune_euclidian(rs_complex, prune_dist**2)
  # END PRUNING

  # CHOOSE MEDIAL AXIS HERE (`example.medial_axis`):
  swaps = list(filter(lambda t: t.dim == medaxdim, swaps.v))

  if 0 < len(swaps):
    faces.append(
      list(map(np.array, 
      rgrid.dual_face(list(old_cell), list(new_cell))))
      )

print(f'Found {len(faces)} faces')
export_obj(example, our_complex, faces)
plt, ax = ourplot.plot_vineyard_results(
  our_complex,
  gridd,
  faces,
  CameraOpt(azim=160, elev=50),
  # only_edges=True,
  # complex_vertices=True,
  skip_grid=True,
  # skip_cube=True
)


NameError: name 'res' is not defined